In [4]:
# !pip install plotly
# !pip install -U kaleido

In [5]:
import glob
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

Hi Matthew, I didn't get vary far because I hit some snags:
- For some reason we have zero `False` entries in the accuracies_df for neither the filter_train_set column or the filter_val_set column
    - This is a a problem because we need this info to create a new col for "baseline accuracy", "major accuracy", "minor "accuracy"
    - If we can't get this then we can't make our table 
    - However, if you do find something wrong in my code that is forcing all `False`s to `True` then we can make it
- If we can do that then create a multi-index on the `t_prior` and `e_prior`
    - If you do this I am not entirely sure that you will even need to do the max on the columns because ease multi-index `(t_prior, e_prior)` pair should only have one value for "baseline accuracy", "major accuracy", "minor "accuracy"
- Serioursly let me know if you have questions

In [ ]:
# Create the dictionary for the dataframe
files = glob.glob('./grid_search/*.txt')
accuracies_list = []

# Go through all the files
for file in files:
    with open(file) as f:
        data_point = {}
        for line in f:

            # stop if we run into the line that starts the predicted states print
            if '~' not in line:
                if line == '\n':
                    pass
                else:
                    # save the params for this point
                    info = line.strip('\n').split(': ')
                    data_point[f'{info[0]}']= f'{info[1]}'
            else:
                break
        accuracies_list.append(data_point)

In [21]:
accuracies_df = pd.DataFrame(accuracies_list)
accuracies_df['Accuracy'] = accuracies_df['Accuracy'].astype(float).round(4)

new_df = accuracies_df.loc[accuracies_df['keys'].isin(['All', 'major', 'minor'])]
new_df['filter_train_set'].value_counts()

filter_train_set
True    557
Name: count, dtype: int64

In [22]:
accuracies_df['keys'].value_counts()

keys
All           439
dorian         80
mixolydian     80
lydian         78
phrygian       76
minor          68
major          50
Name: count, dtype: int64

In [18]:
accuracies_df['filter_train_set'].value_counts()

filter_train_set
True    871
Name: count, dtype: int64

In [20]:
accuracies_df['filter_val_set'].value_counts()

filter_val_set
True    871
Name: count, dtype: int64

In [ ]:
fig = px.scatter_3d(accuracies_df, 
              x='n_chords', 
              y='n_melody', 
              z='Accuracy',
              height=800,)
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

In [6]:
t_prior_labels = accuracies_df['t_prior'].astype(float).value_counts().sort_index().astype(str).index.astype(str).to_list()
e_prior_labels = accuracies_df['e_prior'].astype(float).value_counts().sort_index().astype(str).index.astype(str).to_list()
print(t_prior_labels)
print(type(t_prior_labels[0]))
print(e_prior_labels)
print(type(e_prior_labels[0]))

['0.0', '7.5', '15.0', '20.0', '22.5', '30.0', '50.0', '90.0', '160.0', '230.0', '300.0', '1037.5', '2025.0', '3012.5', '4000.0']
<class 'str'>
['0.0', '7.5', '15.0', '22.5', '30.0', '50.0', '75.0', '150.0', '225.0', '300.0', '1037.5', '2025.0', '3012.5', '4000.0']
<class 'str'>


In [ ]:
px.scatter_3d(accuracies_df, 
              x='t_prior', 
              y='e_prior', 
              z='Accuracy',
              hover_name='Accuracy',
              hover_data=['t_prior', 
                          'e_prior', 
                          'n_chords',
                          'n_melody'],
              category_orders={'t_prior': t_prior_labels,
                               'e_prior':e_prior_labels},
              height=800)

In [75]:
px.scatter_3d(accuracies_df[accuracies_df['Accuracy']>0.32], 
              x='t_prior', 
              y='e_prior', 
              z='Accuracy',
              hover_name='Accuracy',
              hover_data=['t_prior', 
                          'e_prior', 
                          'n_chords',
                          'n_melody'],
              category_orders={'t_prior': t_prior_labels,
                'e_prior':e_prior_labels},
              height=800)

In [15]:
fig = px.scatter(accuracies_df[accuracies_df['Accuracy']>0.32], 
              x='t_prior',  
              y='Accuracy',
              hover_name='Accuracy',
              color='e_prior',
              symbol='e_prior',
              size_max=15,
              symbol_sequence=['circle', 'triangle-up', 'star-triangle-up', 'square',
                               'star-square', 'diamond', 'diamond-tall', 'star-diamond', 
                               'pentagon', 'hexagon', 'octagon', 'cross',
                               'star', 'hexagram'],
              hover_data=['t_prior', 
                          'e_prior', 
                          'n_chords',
                          'n_melody'],
              category_orders={'t_prior': t_prior_labels,
                'e_prior':e_prior_labels},
              height=800,
              width=1200)
fig.update_traces(marker={'size': 10})
pio.write_image(fig, "first_grid_search.pdf", height=800, width=1000)

In [ ]:
px.scatter(accuracies_df[accuracies_df['Accuracy']>0.32], 
              x='t_prior',  
              y='Accuracy',
              hover_name='Accuracy',
              color='e_prior',
              hover_data=['t_prior', 
                          'e_prior', 
                          'n_chords',
                          'n_melody'],
              category_orders={'t_prior': t_prior_labels,
                'e_prior':e_prior_labels},
              height=800)

major
minor
highest overall (not on subsetted dataset)

In [72]:
accuracies_df.loc[accuracies_df['Accuracy'].idxmax()]

Accuracy    0.4597
n_chords         1
n_melody         0
t_prior     3012.5
e_prior       50.0
Name: 53, dtype: object

In [ ]:
print(accuracies_df['Accuracy'].max().round(2))
print(np.floor(accuracies_df['Accuracy'].min()))

0.46
0.0


In [101]:
percs = list(range(int(np.floor(accuracies_df['Accuracy'].min())),
           int(accuracies_df['Accuracy'].max().round(2)*100),
           1
           )
    )
total_len = len(accuracies_df)

previous_perc_above = -1
print('Grid Search Accuracies')
print('---------------------')
for perc in percs[::-1]:
    perc = perc/100.0
    # print(perc)
    perc_len = len(accuracies_df[accuracies_df['Accuracy'] >= perc])
    perc_above = round(perc_len/total_len, 3)

    if perc_above != previous_perc_above:
        print(f"% of grid points >= ({perc:.2f}):\t {round(perc_above, 3)}")
        previous_perc_above = perc_above


Grid Search Accuracies
---------------------
% of grid points >= (0.45):	 0.147
% of grid points >= (0.44):	 0.36
% of grid points >= (0.43):	 0.6
% of grid points >= (0.42):	 0.787
% of grid points >= (0.41):	 0.907
% of grid points >= (0.38):	 0.92
% of grid points >= (0.31):	 0.933
% of grid points >= (0.01):	 0.987
% of grid points >= (0.00):	 1.0
